In [48]:
import pandas as pd
import numpy as np
import requests
import json
import time

# Parameters
rivers_reference = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')

sites = rivers_reference.USGS_ID.tolist()
string = ''

for i in sites:
    string = string+','+str(i)
sites=string[1:]

# URL to load json of data 
# Use format=rmd for readable URL
link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+sites)
r=requests.get(link)
Geodata = json.loads(r.text)

Forecast_df = pd.DataFrame()

for i in range(len(Geodata['value']['timeSeries'])):
    lat = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['latitude'])
    long = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['longitude'])
    baseStation = Geodata['value']['timeSeries'][i]['sourceInfo']['siteName']
    # Search API for Lat Long Position
    r = requests.get('https://api.weather.gov/points/'+lat+','+long)
    json_data = r.json()

    print(baseStation)

    # Retrieve Forecast for Lat Long Position
    forecast_json = requests.get(json_data['properties']['forecast']).json()
    try: 
        if forecast_json['status'] == 500:
            continue
    except:
        KeyError
    forecast = forecast_json['properties']['periods']
    Forecast_df_temp = pd.DataFrame(forecast).drop(columns='number')
    Forecast_df_temp['Lat'] = lat
    Forecast_df_temp['Long'] = long
    Forecast_df_temp['Station'] = baseStation
    Forecast_df = Forecast_df.append(Forecast_df_temp,ignore_index=True)

Forecast_df.shape

E WALKER RV NR BRIDGEPORT, CA
number of forecasts:  14
E WALKER RV NR BRIDGEPORT, CA
number of forecasts:  14
E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEVILLE
number of forecasts:  14
E FK CARSON RV BLW MARKLEEVILLE CK NR MARKLEEVILLE
number of forecasts:  14
LITTLE TRUCKEE R AB BOCA RES NR TRUCKEE CA
number of forecasts:  14
LITTLE TRUCKEE R AB BOCA RES NR TRUCKEE CA
number of forecasts:  14
TRUCKEE R A FARAD CA
number of forecasts:  14
TRUCKEE R A FARAD CA
number of forecasts:  14
TRUCKEE R A FARAD CA
number of forecasts:  14
SAN LORENZO R A BIG TREES CA
number of forecasts:  14
SAN LORENZO R A BIG TREES CA
number of forecasts:  14
SAN LORENZO R A SANTA CRUZ CA
number of forecasts:  14
SAN LORENZO R A SANTA CRUZ CA
number of forecasts:  14
PIT R BL PIT NO 1 PH NR FALL RIVER MILLS CA
number of forecasts:  14
PIT R BL PIT NO 1 PH NR FALL RIVER MILLS CA
number of forecasts:  14
HAT C NR HAT CREEK CA
number of forecasts:  14
HAT C NR HAT CREEK CA
number of forecasts:  14
HAT C NR HAT C

,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,windDirection,icon,shortForecast,detailedForecast,Lat,Long,Station
0,Tonight,2022-03-09T20:00:00-08:00,2022-03-10T06:00:00-08:00,False,13,F,None,10 to 20 mph,NW,"https://api.weather.gov/icons/land/night/snow,...",Chance Light Snow,"A chance of snow after 10pm. Partly cloudy, wi...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
1,Thursday,2022-03-10T06:00:00-08:00,2022-03-10T18:00:00-08:00,True,34,F,None,10 to 20 mph,N,"https://api.weather.gov/icons/land/day/snow,20...",Slight Chance Light Snow then Mostly Sunny,A slight chance of snow before 7am. Mostly sun...,38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
2,Thursday Night,2022-03-10T18:00:00-08:00,2022-03-11T06:00:00-08:00,False,11,F,None,5 to 10 mph,NW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 11. Northwest ...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
3,Friday,2022-03-11T06:00:00-08:00,2022-03-11T18:00:00-08:00,True,50,F,None,5 mph,SW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 50. Southwest wind aro...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
4,Friday Night,2022-03-11T18:00:00-08:00,2022-03-12T06:00:00-08:00,False,20,F,None,5 mph,SW,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 20. Southwest wind ar...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,Monday,2022-03-14T06:00:00-07:00,2022-03-14T18:00:00-07:00,True,56,F,None,13 mph,S,https://api.weather.gov/icons/land/day/rain?si...,Rain,"Rain. Mostly cloudy, with a high near 56.",42.5784422,-124.0581443,"ROGUE RIVER NEAR AGNESS, OR"
1130,Monday Night,2022-03-14T18:00:00-07:00,2022-03-15T06:00:00-07:00,False,42,F,None,7 to 12 mph,SSW,https://api.weather.gov/icons/land/night/rain?...,Rain,"Rain. Mostly cloudy, with a low around 42.",42.5784422,-124.0581443,"ROGUE RIVER NEAR AGNESS, OR"
1131,Tuesday,2022-03-15T06:00:00-07:00,2022-03-15T18:00:00-07:00,True,49,F,None,7 mph,WSW,https://api.weather.gov/icons/land/day/rain?si...,Chance Light Rain,"A chance of rain. Partly sunny, with a high ne...",42.5784422,-124.0581443,"ROGUE RIVER NEAR AGNESS, OR"
1132,Tuesday Night,2022-03-15T18:00:00-07:00,2022-03-16T06:00:00-07:00,False,37,F,None,1 to 7 mph,SW,https://api.weather.gov/icons/land/night/rain?...,Slight Chance Light Rain,"A slight chance of rain. Mostly cloudy, with a...",42.5784422,-124.0581443,"ROGUE RIVER NEAR AGNESS, OR"


In [50]:
df = Forecast_df.drop_duplicates()
df.startTime = 

,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,windDirection,icon,shortForecast,detailedForecast,Lat,Long,Station
0,Tonight,2022-03-09T20:00:00-08:00,2022-03-10T06:00:00-08:00,False,13,F,None,10 to 20 mph,NW,"https://api.weather.gov/icons/land/night/snow,...",Chance Light Snow,"A chance of snow after 10pm. Partly cloudy, wi...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
1,Thursday,2022-03-10T06:00:00-08:00,2022-03-10T18:00:00-08:00,True,34,F,None,10 to 20 mph,N,"https://api.weather.gov/icons/land/day/snow,20...",Slight Chance Light Snow then Mostly Sunny,A slight chance of snow before 7am. Mostly sun...,38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
2,Thursday Night,2022-03-10T18:00:00-08:00,2022-03-11T06:00:00-08:00,False,11,F,None,5 to 10 mph,NW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 11. Northwest ...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
3,Friday,2022-03-11T06:00:00-08:00,2022-03-11T18:00:00-08:00,True,50,F,None,5 mph,SW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 50. Southwest wind aro...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
4,Friday Night,2022-03-11T18:00:00-08:00,2022-03-12T06:00:00-08:00,False,20,F,None,5 mph,SW,https://api.weather.gov/icons/land/night/skc?s...,Clear,"Clear, with a low around 20. Southwest wind ar...",38.3276958,-119.2148759,"E WALKER RV NR BRIDGEPORT, CA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,Monday,2022-03-14T06:00:00-07:00,2022-03-14T18:00:00-07:00,True,56,F,None,13 mph,S,https://api.weather.gov/icons/land/day/rain?si...,Rain,"Rain. Mostly cloudy, with a high near 56.",42.5784422,-124.0581443,"ROGUE RIVER NEAR AGNESS, OR"
1102,Monday Night,2022-03-14T18:00:00-07:00,2022-03-15T06:00:00-07:00,False,42,F,None,7 to 12 mph,SSW,https://api.weather.gov/icons/land/night/rain?...,Rain,"Rain. Mostly cloudy, with a low around 42.",42.5784422,-124.0581443,"ROGUE RIVER NEAR AGNESS, OR"
1103,Tuesday,2022-03-15T06:00:00-07:00,2022-03-15T18:00:00-07:00,True,49,F,None,7 mph,WSW,https://api.weather.gov/icons/land/day/rain?si...,Chance Light Rain,"A chance of rain. Partly sunny, with a high ne...",42.5784422,-124.0581443,"ROGUE RIVER NEAR AGNESS, OR"
1104,Tuesday Night,2022-03-15T18:00:00-07:00,2022-03-16T06:00:00-07:00,False,37,F,None,1 to 7 mph,SW,https://api.weather.gov/icons/land/night/rain?...,Slight Chance Light Rain,"A slight chance of rain. Mostly cloudy, with a...",42.5784422,-124.0581443,"ROGUE RIVER NEAR AGNESS, OR"
